# Analysis of APT data (statistical distributions)

**System**
* Alloy composition: $Al_{0.1}FeNiCrCo$
* Lattice: FCC
* Lattice constant: 3.57A

**APT measurement**
* Detector efficiency: 33%
* position uncertainty (standard deviation): 5A (in x-, y-directions) and 3A in z-direction

## Setup

In [1]:
# Notebook settings

# code development settings (automatically reload modified modules)
%load_ext autoreload
%autoreload 2

# plotting backend
%matplotlib inline

In [2]:
# basic libraries

import os, sys
import re
import copy
import glob
from itertools import product
import pickle
from collections import Counter
import numpy as np
#import scipy.io
from scipy.stats import multinomial, chi2
from scipy.optimize import fmin
from sklearn.neighbors import NearestNeighbors

# plotting
import matplotlib.pyplot as plt

# data handling
# import pyUSID as usid
# import h5py
# import pandas as pd
# import xarray as xr

# parallelization
# from subprocess import Popen, PIPE
import multiprocessing as mp

In [3]:
# local library
sys.path.append('../../../force_fields/statmechlib')
from statmechlib.read_write import read_lattice_mc, read_modeldef, write_modeldef
from statmechlib.preprocessing import get_stats_latt, cfg_replicate, latt_to_real_coords, add_experimental_noise
from statmechlib.preprocessing import real_coords, select_core, get_knn_stats
from statmechlib.forcefields import sd2, get_chi2_two, get_s2_two, data_lists, loss_sd2_hist
from statmechlib.forcefields import make_reference_matrices, make_target_matrices

In [5]:
# project directories
data_raw = '../data/interim'
src_dir = '../src/utils'
sim_dir = '../sim/enth_model'
sim_dir_m1 = '../sim/m1'
sim_dir_m2 = '../sim/m2'
sim_dir_m3 = '../sim/m3'

In [6]:
sys.path.append(src_dir)
from utils import *

## Read target data

In [7]:
# compositions, concentrations, atom types
element = ['Al','Fe','Ni','Cr','Co']
ntype = len(element)

# numbers of KNNs of a particular atom i of type ti
knn = list(range(1,13))
#knn += [20, 40, 100, 200, 500, 1000]

In [8]:
targets = {}
for k in knn:
    targets[k] = read_pairwise(os.path.join(data_raw, f'RamaTest_Clean_PatternK{k}_PairWise'), k, ntype)

In [9]:
# count statistics and concentrations
counts = np.sum(targets[knn[5]], axis=2)[:,0]
assert np.array_equal(counts, np.sum(targets[knn[0]], axis=2)[:,0])

conc = counts/np.sum(counts)
tot_counts = np.sum(counts)

In [14]:
# kksum = 0
# for kk in targets[1][:]:
#     ks = sum([i*p for i, p in enumerate(kk[-1])])
#     print(ks)
#     kksum += ks
# print(kksum)

## Generate random distributions (null hypothesis)

* Assume that there are no interactioins between particles and for each KNN generate multinomial distributions with parameters (probabilities) set to mole fractions.

In [15]:
null_reference = {}
null_counts = {}
for k in knn:
    null_reference[k] = get_binomials(k, conc)
    null_counts[k] = null_reference[k]*tot_counts

## Read reference data from MC simulations

In [16]:
# read in a set of trajectories
#trj_dirs = ['txinf', 'tx500', 'tx700', 'tx1000']
#trj_dirs = ['1x700']
trj_dirs = ['3x700']

In [18]:
trjs = {}
for key in trj_dirs:
    # read trajectory info
    trjs[key] = read_lattice_mc(os.path.join(sim_dir_m3, key), verbose=1)

Reading ../sim/m3/3x700/lg.hst
Reading ../sim/m3/3x700/lg.run
Reading ../sim/m3/3x700/lg.xyz
Reading ../sim/m3/3x700/lg.mld


In [19]:
trjs['3x700']['ref_params']

array([ 0.  , -2.97, -7.  , -1.05, -4.  ,  0.  , -0.78, -0.06, -0.48,
        0.  , -0.24, -0.17,  0.  ,  0.04,  0.  ])

## Add experimental noise

In [20]:
unknown_frac = counts[-1]/np.sum(counts)

# analyze configurations
for key, trj in trjs.items():
    if key not in ['3x700']:
        continue
    print('trj:', key)

    # convert from lattice to real coordinates
    trj = latt_to_real_coords(trj, 3.57/2.0)

    # model of experimental noise
    xyzs, typs = add_experimental_noise(trj, loss_rate=0.67, disp=np.array([5.0, 5.0, 3.0]), unknown_frac=0.0)
    trjs[key]['xyz_noise'] = xyzs
    trjs[key]['atom_type_noise'] = typs

trj: 3x700


In [21]:
# 1. replicate configurations to avoid dealing with periodic boundary conditions
# 2. convert to real coordinates
# 3. select core
for key, trj in trjs.items():
    if key not in ['3x700']:
        continue

    print('trj:', key)
    xyzs = []
    boxs = []
    tis = []
    for xyz, box, ti in zip(trj['xyz_noise'], trj['box'], trj['atom_type_noise']):
        xyz_big, box_big = cfg_replicate(xyz, box, vec_a=2, vec_b=2, vec_c=2)
        xyzs.append(xyz_big)
        boxs.append(box_big)
        tis.append(np.tile(ti, 2*2*2))                                          

    trj['xyz_noise'] = xyzs
    trj['box_noise'] = boxs
    trj['atom_type_noise'] = tis
    
    trj['xyz_noise'] = real_coords(trj['xyz_noise'], trj['box_noise'])
    
    trj['xyz_core'], trj['atom_type_core'] = select_core(trj['xyz_noise'], trj['box_noise'], trj['atom_type_noise'])

trj: 3x700


In [22]:
# with open('../data/working/trjs.pickle', 'wb') as fo:
#     pickle.dump(trjs, fo)

### Neighbor statistics

In [23]:
knns = list(range(1,13))
#knns += [20, 40, 100, 200]

# create nighbor model with the maximum of nearest neighbors (1000). KDTree should make this Nlog(N)
neigh = NearestNeighbors(n_neighbors=knns[-1], algorithm='kd_tree', n_jobs=4)

In [24]:
for key, trj in trjs.items():
    if key not in ['3x700']:
        continue

    print('trj:', key)

    knn_stats = []
    for it, (cor, xyz, ti_core, ti) in enumerate(zip(trj['xyz_core'], trj['xyz_noise'], trj['atom_type_core'], trj['atom_type_noise'])):
        print(it, len(ti_core), len(ti))
        if it < 2000:
            continue
        neigh.fit(xyz)
        dist, ind = neigh.kneighbors(cor, n_neighbors=knns[-1]+1)
        assert np.array_equal(dist[:,0], np.zeros(dist.shape[0])), "First neighbor is not center"
        knn_stats.append(get_knn_stats(ind, ti_core, ti, knns=knns))

    trj['knn'] = knn_stats

trj: 3x700
0 470 3760
1 487 3896
2 445 3560
3 435 3480
4 451 3608
5 457 3656
6 447 3576
7 447 3576
8 459 3672
9 469 3752
10 467 3736
11 442 3536
12 438 3504
13 442 3536
14 432 3456
15 429 3432
16 448 3584
17 432 3456
18 430 3440
19 474 3792
20 438 3504
21 426 3408
22 415 3320
23 433 3464
24 472 3776
25 442 3536
26 462 3696
27 428 3424
28 462 3696
29 448 3584
30 431 3448
31 485 3880
32 461 3688
33 461 3688
34 465 3720
35 424 3392
36 452 3616
37 444 3552
38 434 3472
39 431 3448
40 456 3648
41 438 3504
42 476 3808
43 443 3544
44 476 3808
45 460 3680
46 502 4016
47 449 3592
48 448 3584
49 465 3720
50 435 3480
51 482 3856
52 446 3568
53 476 3808
54 445 3560
55 453 3624
56 442 3536
57 446 3568
58 483 3864
59 432 3456
60 479 3832
61 477 3816
62 465 3720
63 446 3568
64 480 3840
65 482 3856
66 453 3624
67 442 3536
68 448 3584
69 436 3488
70 430 3440
71 437 3496
72 454 3632
73 456 3648
74 452 3616
75 453 3624
76 455 3640
77 442 3536
78 439 3512
79 452 3616
80 463 3704
81 474 3792
82 442 3536
83 

700 478 3824
701 466 3728
702 442 3536
703 430 3440
704 474 3792
705 439 3512
706 458 3664
707 445 3560
708 471 3768
709 457 3656
710 436 3488
711 459 3672
712 451 3608
713 451 3608
714 433 3464
715 437 3496
716 476 3808
717 436 3488
718 446 3568
719 447 3576
720 450 3600
721 477 3816
722 432 3456
723 460 3680
724 417 3336
725 436 3488
726 468 3744
727 462 3696
728 436 3488
729 429 3432
730 461 3688
731 444 3552
732 451 3608
733 444 3552
734 455 3640
735 437 3496
736 455 3640
737 467 3736
738 477 3816
739 440 3520
740 459 3672
741 474 3792
742 404 3232
743 448 3584
744 407 3256
745 433 3464
746 466 3728
747 448 3584
748 450 3600
749 434 3472
750 437 3496
751 461 3688
752 449 3592
753 444 3552
754 454 3632
755 437 3496
756 467 3736
757 470 3760
758 447 3576
759 465 3720
760 453 3624
761 405 3240
762 451 3608
763 440 3520
764 441 3528
765 482 3856
766 467 3736
767 482 3856
768 450 3600
769 440 3520
770 426 3408
771 447 3576
772 460 3680
773 435 3480
774 472 3776
775 451 3608
776 436 3488

1940 463 3704
1941 434 3472
1942 490 3920
1943 461 3688
1944 464 3712
1945 448 3584
1946 417 3336
1947 418 3344
1948 429 3432
1949 410 3280
1950 449 3592
1951 429 3432
1952 451 3608
1953 482 3856
1954 442 3536
1955 436 3488
1956 448 3584
1957 439 3512
1958 462 3696
1959 439 3512
1960 462 3696
1961 447 3576
1962 464 3712
1963 437 3496
1964 440 3520
1965 464 3712
1966 439 3512
1967 494 3952
1968 437 3496
1969 455 3640
1970 444 3552
1971 458 3664
1972 453 3624
1973 450 3600
1974 434 3472
1975 452 3616
1976 452 3616
1977 434 3472
1978 442 3536
1979 451 3608
1980 457 3656
1981 439 3512
1982 466 3728
1983 431 3448
1984 444 3552
1985 446 3568
1986 452 3616
1987 474 3792
1988 462 3696
1989 470 3760
1990 409 3272
1991 452 3616
1992 466 3728
1993 473 3784
1994 453 3624
1995 446 3568
1996 449 3592
1997 447 3576
1998 459 3672
1999 430 3440
2000 449 3592
2001 459 3672
2002 470 3760
2003 433 3464
2004 436 3488
2005 466 3728
2006 476 3808
2007 437 3496
2008 472 3776
2009 447 3576
2010 462 3696
2011 4

2527 433 3464
2528 447 3576
2529 443 3544
2530 448 3584
2531 465 3720
2532 477 3816
2533 449 3592
2534 439 3512
2535 471 3768
2536 455 3640
2537 478 3824
2538 438 3504
2539 475 3800
2540 437 3496
2541 462 3696
2542 428 3424
2543 439 3512
2544 459 3672
2545 449 3592
2546 452 3616
2547 428 3424
2548 441 3528
2549 472 3776
2550 484 3872
2551 431 3448
2552 466 3728
2553 450 3600
2554 472 3776
2555 409 3272
2556 466 3728
2557 462 3696
2558 447 3576
2559 433 3464
2560 432 3456
2561 471 3768
2562 436 3488
2563 467 3736
2564 456 3648
2565 447 3576
2566 423 3384
2567 437 3496
2568 456 3648
2569 439 3512
2570 462 3696
2571 474 3792
2572 491 3928
2573 438 3504
2574 421 3368
2575 432 3456
2576 438 3504
2577 447 3576
2578 436 3488
2579 490 3920
2580 453 3624
2581 459 3672
2582 446 3568
2583 428 3424
2584 431 3448
2585 457 3656
2586 451 3608
2587 464 3712
2588 469 3752
2589 465 3720
2590 429 3432
2591 451 3608
2592 430 3440
2593 449 3592
2594 461 3688
2595 466 3728
2596 448 3584
2597 439 3512
2598 4

3113 475 3800
3114 453 3624
3115 467 3736
3116 435 3480
3117 440 3520
3118 428 3424
3119 479 3832
3120 455 3640
3121 474 3792
3122 461 3688
3123 457 3656
3124 458 3664
3125 459 3672
3126 440 3520
3127 455 3640
3128 438 3504
3129 472 3776
3130 457 3656
3131 462 3696
3132 455 3640
3133 438 3504
3134 443 3544
3135 447 3576
3136 421 3368
3137 476 3808
3138 452 3616
3139 428 3424
3140 457 3656
3141 443 3544
3142 462 3696
3143 447 3576
3144 479 3832
3145 459 3672
3146 472 3776
3147 443 3544
3148 438 3504
3149 439 3512
3150 459 3672
3151 443 3544
3152 457 3656
3153 456 3648
3154 447 3576
3155 490 3920
3156 451 3608
3157 459 3672
3158 464 3712
3159 451 3608
3160 461 3688
3161 459 3672
3162 435 3480
3163 457 3656
3164 440 3520
3165 423 3384
3166 431 3448
3167 467 3736
3168 463 3704
3169 415 3320
3170 436 3488
3171 429 3432
3172 432 3456
3173 472 3776
3174 460 3680
3175 456 3648
3176 451 3608
3177 469 3752
3178 457 3656
3179 455 3640
3180 438 3504
3181 459 3672
3182 440 3520
3183 458 3664
3184 4

...

9563 429 3432
9564 428 3424
9565 443 3544
9566 481 3848
9567 455 3640
9568 428 3424
9569 488 3904
9570 439 3512
9571 462 3696
9572 452 3616
9573 467 3736
9574 430 3440
9575 457 3656
9576 429 3432
9577 440 3520
9578 452 3616
9579 462 3696
9580 433 3464
9581 442 3536
9582 458 3664
9583 455 3640
9584 434 3472
9585 447 3576
9586 458 3664
9587 434 3472
9588 482 3856
9589 457 3656
9590 434 3472
9591 466 3728
9592 473 3784
9593 426 3408
9594 451 3608
9595 439 3512
9596 462 3696
9597 443 3544
9598 473 3784
9599 446 3568
9600 460 3680
9601 441 3528
9602 469 3752
9603 435 3480
9604 472 3776
9605 425 3400
9606 504 4032
9607 453 3624
9608 422 3376
9609 429 3432
9610 462 3696
9611 467 3736
9612 461 3688
9613 475 3800
9614 463 3704
9615 439 3512
9616 450 3600
9617 444 3552
9618 477 3816
9619 441 3528
9620 433 3464
9621 466 3728
9622 440 3520
9623 476 3808
9624 461 3688
9625 455 3640
9626 455 3640
9627 480 3840
9628 451 3608
9629 471 3768
9630 505 4040
9631 460 3680
9632 457 3656
9633 447 3576
9634 4

10139 450 3600
10140 444 3552
10141 455 3640
10142 438 3504
10143 465 3720
10144 462 3696
10145 460 3680
10146 460 3680
10147 462 3696
10148 450 3600
10149 490 3920
10150 434 3472
10151 440 3520
10152 493 3944
10153 459 3672
10154 456 3648
10155 420 3360
10156 454 3632
10157 423 3384
10158 434 3472
10159 427 3416
10160 460 3680
10161 465 3720
10162 467 3736
10163 444 3552
10164 457 3656
10165 468 3744
10166 439 3512
10167 443 3544
10168 455 3640
10169 449 3592
10170 457 3656
10171 458 3664
10172 480 3840
10173 422 3376
10174 452 3616
10175 429 3432
10176 440 3520
10177 486 3888
10178 441 3528
10179 476 3808
10180 459 3672
10181 476 3808
10182 464 3712
10183 470 3760
10184 445 3560
10185 447 3576
10186 445 3560
10187 444 3552
10188 459 3672
10189 482 3856
10190 445 3560
10191 469 3752
10192 442 3536
10193 438 3504
10194 414 3312
10195 445 3560
10196 468 3744
10197 454 3632
10198 438 3504
10199 453 3624
10200 432 3456
10201 467 3736
10202 473 3784
10203 453 3624
10204 453 3624
10205 477 

10686 454 3632
10687 493 3944
10688 457 3656
10689 470 3760
10690 459 3672
10691 486 3888
10692 457 3656
10693 471 3768
10694 423 3384
10695 473 3784
10696 442 3536
10697 477 3816
10698 443 3544
10699 469 3752
10700 454 3632
10701 463 3704
10702 467 3736
10703 448 3584
10704 463 3704
10705 468 3744
10706 471 3768
10707 447 3576
10708 451 3608
10709 452 3616
10710 437 3496
10711 477 3816
10712 448 3584
10713 468 3744
10714 442 3536
10715 460 3680
10716 464 3712
10717 433 3464
10718 441 3528
10719 442 3536
10720 481 3848
10721 447 3576
10722 419 3352
10723 478 3824
10724 434 3472
10725 457 3656
10726 449 3592
10727 448 3584
10728 466 3728
10729 427 3416
10730 483 3864
10731 449 3592
10732 464 3712
10733 442 3536
10734 435 3480
10735 413 3304
10736 473 3784
10737 430 3440
10738 438 3504
10739 482 3856
10740 452 3616
10741 470 3760
10742 456 3648
10743 457 3656
10744 448 3584
10745 443 3544
10746 447 3576
10747 473 3784
10748 474 3792
10749 439 3512
10750 445 3560
10751 441 3528
10752 462 

11234 457 3656
11235 447 3576
11236 455 3640
11237 451 3608
11238 447 3576
11239 437 3496
11240 458 3664
11241 478 3824
11242 440 3520
11243 435 3480
11244 448 3584
11245 457 3656
11246 452 3616
11247 457 3656
11248 444 3552
11249 445 3560
11250 454 3632
11251 464 3712
11252 462 3696
11253 455 3640
11254 466 3728
11255 458 3664
11256 447 3576
11257 451 3608
11258 480 3840
11259 464 3712
11260 451 3608
11261 467 3736
11262 436 3488
11263 451 3608
11264 452 3616
11265 459 3672
11266 463 3704
11267 427 3416
11268 445 3560
11269 432 3456
11270 454 3632
11271 438 3504
11272 462 3696
11273 452 3616
11274 464 3712
11275 464 3712
11276 476 3808
11277 479 3832
11278 481 3848
11279 454 3632
11280 452 3616
11281 428 3424
11282 484 3872
11283 445 3560
11284 468 3744
11285 437 3496
11286 438 3504
11287 460 3680
11288 471 3768
11289 443 3544
11290 436 3488
11291 442 3536
11292 443 3544
11293 449 3592
11294 474 3792
11295 475 3800
11296 433 3464
11297 460 3680
11298 450 3600
11299 439 3512
11300 405 

11781 436 3488
11782 419 3352
11783 461 3688
11784 455 3640
11785 460 3680
11786 476 3808
11787 467 3736
11788 454 3632
11789 450 3600
11790 456 3648
11791 446 3568
11792 460 3680
11793 438 3504
11794 453 3624
11795 461 3688
11796 473 3784
11797 430 3440
11798 461 3688
11799 463 3704
11800 415 3320
11801 473 3784
11802 443 3544
11803 437 3496
11804 431 3448
11805 415 3320
11806 446 3568
11807 441 3528
11808 463 3704
11809 434 3472
11810 446 3568
11811 464 3712
11812 442 3536
11813 454 3632
11814 426 3408
11815 413 3304
11816 460 3680
11817 476 3808
11818 457 3656
11819 486 3888
11820 486 3888
11821 452 3616
11822 464 3712
11823 464 3712
11824 473 3784
11825 458 3664
11826 430 3440
11827 502 4016
11828 439 3512
11829 454 3632
11830 453 3624
11831 421 3368
11832 428 3424
11833 509 4072
11834 427 3416
11835 432 3456
11836 422 3376
11837 454 3632
11838 448 3584
11839 453 3624
11840 464 3712
11841 469 3752
11842 458 3664
11843 481 3848
11844 477 3816
11845 443 3544
11846 411 3288
11847 480 

In [25]:
ti[ind[2]], ti_core[2], np.array_equal(dist[:,0], np.zeros(dist.shape[0]))

(array([4, 2, 2, 2, 3, 4, 3, 5, 4, 5, 2, 2, 5]), 4, True)

In [26]:
knn_stats[-1][0]

array([[[ 13,   0],
        [ 11,   2],
        [  7,   6],
        [ 12,   1],
        [  9,   4]],

       [[106,   1],
        [ 92,  15],
        [ 71,  36],
        [ 83,  24],
        [ 76,  31]],

       [[132,   5],
        [103,  34],
        [ 92,  45],
        [113,  24],
        [108,  29]],

       [[ 93,   2],
        [ 70,  25],
        [ 71,  24],
        [ 66,  29],
        [ 80,  15]],

       [[101,   4],
        [ 81,  24],
        [ 70,  35],
        [ 91,  14],
        [ 77,  28]]])

In [28]:
trjs['3x700']['atom_type_core'][1]

array([5, 3, 5, 2, 4, 5, 2, 2, 4, 3, 2, 5, 4, 5, 3, 3, 2, 4, 3, 2, 5, 3,
       4, 5, 3, 1, 4, 5, 5, 5, 2, 4, 2, 3, 5, 3, 2, 3, 1, 4, 2, 5, 5, 5,
       3, 3, 4, 1, 4, 4, 4, 3, 2, 2, 4, 4, 3, 4, 5, 2, 2, 3, 5, 2, 4, 4,
       5, 2, 4, 4, 2, 4, 4, 5, 4, 2, 3, 4, 3, 3, 3, 5, 3, 4, 3, 2, 3, 5,
       4, 5, 5, 4, 1, 4, 5, 5, 2, 4, 3, 2, 2, 5, 4, 2, 4, 4, 5, 2, 2, 2,
       5, 5, 2, 2, 5, 4, 1, 5, 2, 3, 5, 3, 5, 5, 4, 4, 1, 4, 3, 3, 3, 3,
       3, 2, 3, 4, 5, 5, 4, 5, 4, 3, 2, 5, 4, 5, 4, 2, 2, 2, 2, 5, 2, 2,
       4, 4, 3, 4, 3, 2, 5, 3, 5, 4, 4, 3, 4, 4, 4, 5, 1, 2, 3, 4, 2, 4,
       4, 3, 5, 4, 4, 3, 2, 3, 4, 3, 4, 3, 4, 3, 3, 4, 3, 4, 3, 5, 2, 5,
       3, 4, 2, 5, 4, 4, 5, 3, 4, 5, 4, 3, 3, 4, 5, 3, 4, 4, 5, 5, 3, 2,
       3, 4, 3, 3, 5, 4, 3, 1, 3, 3, 3, 2, 3, 4, 5, 2, 2, 5, 5, 3, 3, 2,
       4, 3, 4, 3, 5, 3, 5, 4, 3, 2, 3, 2, 2, 4, 3, 3, 4, 4, 2, 4, 5, 4,
       3, 4, 2, 3, 5, 3, 2, 5, 2, 1, 5, 5, 5, 2, 4, 4, 3, 5, 3, 2, 5, 2,
       5, 5, 5, 5, 5, 4, 2, 4, 3, 4, 4, 5, 4, 5, 5,

In [29]:
# with open('../data/working/trjs_knn.pickle', 'wb') as fo:
#     pickle.dump(trjs, fo)

In [30]:
trjs['3x700'].keys()

dict_keys(['energy', 'interaction_stats', 'temp', 'box_latt', 'xyz_latt', 'atom_type', 'ref_params', 'box', 'xyz', 'xyz_noise', 'atom_type_noise', 'box_noise', 'xyz_core', 'atom_type_core', 'knn'])

In [32]:
knn_all = {}
ppp_all = {}
conc_all = []

for key in ['3x700']:
    hists = trjs[key]['knn']
    print(len(hists), len(hists[0]), hists[0][0].shape)
    
    # sum histograms
    print(key)
    knn_trj = []
    ppp_trj = []
    for i, k in enumerate(knns):
        hhh = np.zeros((ntype, ntype, k+1), dtype=float)
        for hst in hists:
            hhh += hst[i]
        knn_trj.append(hhh)
        hsum = np.sum(hhh, axis=2)
        #print(i, hsum[:,0])
        if i == 0:
            counts_all = hsum[:,1]
            conc_all = counts_all/np.sum(counts_all)
        ppp = hhh/hsum[:,:,None]
        ppp_trj.append(ppp)
    
    knn_all[key] = knn_trj
    ppp_all[key] = ppp_trj
    # get probability distributions
tot_counts_all = np.sum(counts_all)
print(conc_all, counts_all/10001, tot_counts_all)
print(conc, counts, tot_counts)

10001 12 (5, 5, 2)
3x700
[0.02546358 0.25093573 0.25065104 0.23394963 0.23900003] [ 11.52814719 113.60633937 113.47745225 105.91620838 108.20267973] 4527761.0
[0.02557753 0.25074298 0.25070882 0.23409538 0.23887529] [ 30699 300950 300909 280969 286706] 1200233


In [33]:
trjs['3x700']['knn'][0][0].shape

(5, 5, 2)

In [41]:
#  optimize only unlike parameters

select = []
for i in range(1, ntype+1):
    for j in range(i, ntype+1):
        if i == 1 and (j == 2 or j == 3 or j == 5):
            select.append(True)
        else:
            select.append(False)
#         #print(i,j)
#         if i == j:
#             select.append(False)
#         else:
#             select.append(True)

#         hist_ave = average_histogram(pars_in, params_list, X_list[key], df_est[key][0], hist_list[key])
#         cb1 = np.sqrt(hist_ave).dot(np.sqrt(hist_targ[key]))
#         s2 = np.arccos(cb1)**2

In [42]:
sum(select)

3

In [43]:
X_list, beta_ref, pars_list, ene_ref, hist_list = data_lists(trjs, minval=-10000)

In [44]:
len(trjs['3x700']['temp']), len(beta_ref[0]), len(hist_list[0])

(12001, 10000, 10000)

In [47]:
pars_list, knns

([array([ 0.  , -2.97, -7.  , -1.05, -4.  ,  0.  , -0.78, -0.06, -0.48,
          0.  , -0.24, -0.17,  0.  ,  0.04,  0.  ])],
 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12])

In [48]:
X_ref, pars_ref, hist_ref = make_reference_matrices(hist_list, X_list, pars_list, select, knn_max=12)

In [49]:
targets[1].shape

(5, 5, 2)

In [53]:
hist_targ = make_target_matrices(targets)[:12,:,:]

In [54]:
X_ref[0].shape, pars_ref[0].shape, beta_ref[0].shape, hist_ref[0].shape, hist_targ.shape

((10000, 3), (3,), (10000,), (10000, 12, 5, 5), (12, 5, 5))

In [56]:
pars_in = pars_ref[0]

In [57]:
pars_in

array([-2.97, -7.  , -4.  ])

In [58]:
output = fmin(loss_sd2_hist, pars_in, args=(X_ref[0], pars_ref[0], beta_ref[0], hist_ref[0], hist_targ))

0
[0.02537256 0.25019517 0.25549522 0.23141514 0.2375219 ]
[0.02472393 0.24818398 0.26039936 0.22958403 0.2371087 ]
0.9999823079846188
1
[0.02555675 0.24963558 0.25134852 0.23357041 0.23988874]
[0.02535637 0.25039375 0.25112145 0.23527496 0.23785346]
0.999995772830877
2
[0.02593644 0.25129409 0.25008151 0.23402369 0.23866426]
[0.02657282 0.25029826 0.24985295 0.23417711 0.23909886]
0.9999974400310616
3
[0.02505561 0.25135956 0.25071377 0.23386558 0.23900548]
[0.02528393 0.25189968 0.24920543 0.23405073 0.23956024]
0.9999982793740678
4
[0.02524047 0.2517005  0.25054065 0.23439325 0.23812512]
[0.02555231 0.25130622 0.251512   0.23550955 0.23611993]
0.9999961917289245
0
[0.0252598  0.24980483 0.25357385 0.23301556 0.23834597]
[0.02482166 0.24798853 0.2603505  0.23067527 0.23616404]
0.9999695833187903
1
[0.02558404 0.24967915 0.25108885 0.23419758 0.23945038]
[0.0251487  0.24989035 0.25074431 0.23483635 0.2393803 ]
0.9999987645552888
2
[0.02578619 0.25102003 0.25013263 0.23442553 0.2386356

0
[0.02531032 0.2508127  0.25423644 0.23155264 0.2380879 ]
[0.02495195 0.24948152 0.25761425 0.23158735 0.23636492]
0.9999913379865336
1
[0.0254456  0.24992834 0.25101847 0.23419581 0.23941178]
[0.02516642 0.24964944 0.25077311 0.23493493 0.2394761 ]
0.999999252807755
2
[0.02582706 0.25126434 0.24998775 0.23407162 0.23884923]
[0.02608042 0.25082999 0.24989172 0.23408406 0.23911382]
0.9999995555826298
3
[0.02519918 0.25113856 0.25066902 0.2339939  0.23899933]
[0.02522402 0.2510971  0.24997182 0.23437698 0.23933008]
0.9999996178465463
4
[0.02536644 0.25136372 0.25057764 0.23407137 0.23862083]
[0.02522212 0.25124925 0.2508028  0.23519564 0.2375302 ]
0.9999985673699541
0
[0.02517801 0.25088021 0.25389698 0.23183679 0.23820801]
[0.02511948 0.24896111 0.25746302 0.23173859 0.23671781]
0.9999907496191058
1
[0.02544753 0.24994567 0.25113932 0.23409306 0.23937441]
[0.02515748 0.24979612 0.25082857 0.23482021 0.23939762]
0.999999242903542
2
[0.02578405 0.2512796  0.25005615 0.23400787 0.23887233

[0.02536745 0.25149892 0.25099405 0.23515989 0.23697969]
0.9999977481297684
0
[0.02528066 0.25091659 0.253837   0.2318925  0.23807325]
[0.02478257 0.24909606 0.25764357 0.23218997 0.23628783]
0.9999882937523694
1
[0.02545834 0.24983024 0.25105583 0.23429484 0.23936075]
[0.02516099 0.24979764 0.2506948  0.23482173 0.23952484]
0.9999993358504322
2
[0.02579939 0.25123537 0.2500291  0.23409336 0.23884278]
[0.02617137 0.25100944 0.24955784 0.23418376 0.23907759]
0.9999991646442862
3
[0.02523916 0.2511655  0.25062752 0.23406722 0.2389006 ]
[0.02513516 0.25117148 0.24985176 0.23452979 0.23931181]
0.9999994431392912
4
[0.02540416 0.25126713 0.25054445 0.23403858 0.23874567]
[0.02522026 0.25119949 0.25092604 0.2354377  0.23721652]
0.9999974875227706
0
[0.02531204 0.25080323 0.2542517  0.23154776 0.23808526]
[0.02495195 0.24948152 0.25761425 0.23158735 0.23636492]
0.9999913994082417
1
[0.02544564 0.2499263  0.25101863 0.23419594 0.23941349]
[0.02516642 0.24964944 0.25077311 0.23493493 0.2394761 

0
[0.02538442 0.25014955 0.25553638 0.23139672 0.23753293]
[0.02472393 0.24818398 0.26039936 0.22958403 0.2371087 ]
0.999982549071456
1
[0.02555792 0.24963373 0.25134102 0.23356797 0.23989936]
[0.02535637 0.25039375 0.25112145 0.23527496 0.23785346]
0.9999957436903567
2
[0.02594084 0.25127685 0.25008077 0.23401962 0.23868192]
[0.02657282 0.25029826 0.24985295 0.23417711 0.23909886]
0.9999974910776033
3
[0.0250509  0.25134645 0.25071729 0.23386061 0.23902474]
[0.02528393 0.25189968 0.24920543 0.23405073 0.23956024]
0.999998266115325
4
[0.02523804 0.25169457 0.25054559 0.23440488 0.23811691]
[0.02555231 0.25130622 0.251512   0.23550955 0.23611993]
0.9999962223408287
0
[0.02526192 0.24979444 0.25360635 0.23299692 0.23834037]
[0.02482166 0.24798853 0.2603505  0.23067527 0.23616404]
0.9999698676952994
1
[0.02558373 0.24966852 0.25108651 0.23420118 0.23946006]
[0.0251487  0.24989035 0.25074431 0.23483635 0.2393803 ]
0.999998766044723
2
[0.0257881  0.25101384 0.25013862 0.23442003 0.2386394 ]

0
[0.02538375 0.25014126 0.25554238 0.23140107 0.23753154]
[0.02472393 0.24818398 0.26039936 0.22958403 0.2371087 ]
0.9999825901770778
1
[0.025557   0.24963136 0.25133728 0.23356879 0.23990556]
[0.02535637 0.25039375 0.25112145 0.23527496 0.23785346]
0.9999957327392157
2
[0.02594111 0.25127298 0.25007738 0.23401945 0.23868908]
[0.02657282 0.25029826 0.24985295 0.23417711 0.23909886]
0.9999975002920845
3
[0.02504972 0.25134189 0.25072133 0.23385999 0.23902708]
[0.02528393 0.25189968 0.24920543 0.23405073 0.23956024]
0.9999982559036954
4
[0.02523662 0.25169656 0.25054672 0.23440562 0.23811448]
[0.02555231 0.25130622 0.251512   0.23550955 0.23611993]
0.9999962241898492
0
[0.02525983 0.24979593 0.2536098  0.23300015 0.23833429]
[0.02482166 0.24798853 0.2603505  0.23067527 0.23616404]
0.9999699027316984
1
[0.0255833  0.24966324 0.25108502 0.2342036  0.23946484]
[0.0251487  0.24989035 0.25074431 0.23483635 0.2393803 ]
0.9999987684135233
2
[0.02578794 0.25101102 0.25013992 0.23441999 0.238641

...

0
[0.02483296 0.25091006 0.25383082 0.23185634 0.23856983]
[0.02575151 0.2491849  0.25593668 0.23184942 0.23727749]
0.9999912896241292
1
[0.02549542 0.24986234 0.25110208 0.23412758 0.23941259]
[0.02526243 0.2497594  0.25070821 0.23478862 0.23948134]
0.9999994146015553
2
[0.02578729 0.25138328 0.25004497 0.23390491 0.23887956]
[0.02601391 0.25059585 0.25032485 0.23403209 0.2390333 ]
0.9999993832007589
3
[0.02523481 0.25112399 0.2505906  0.23392309 0.2391275 ]
[0.02514448 0.25098272 0.25003179 0.23443026 0.23941075]
0.9999996144105632
4
[0.02544964 0.25141726 0.25058398 0.23420967 0.23833945]
[0.02532559 0.25094443 0.2504428  0.23499651 0.23829067]
0.999999471920834
0
[0.02480634 0.25088331 0.25384081 0.2317961  0.23867344]
[0.02588304 0.24950596 0.25548878 0.23188052 0.23724171]
0.9999909203273015
1
[0.02548895 0.24987489 0.25110165 0.23413434 0.23940017]
[0.02529988 0.24980423 0.25061167 0.23475688 0.23952733]
0.9999994868443624
2
[0.02577583 0.25139825 0.25006342 0.23392694 0.2388355

0
[0.02542515 0.25004003 0.25562861 0.23130681 0.2375994 ]
[0.02472393 0.24818398 0.26039936 0.22958403 0.2371087 ]
0.9999830565325551
1
[0.02556556 0.24963149 0.25132429 0.23357115 0.23990751]
[0.02535637 0.25039375 0.25112145 0.23527496 0.23785346]
0.9999957184752554
2
[0.02594877 0.25123391 0.25008896 0.23404788 0.23868048]
[0.02657282 0.25029826 0.24985295 0.23417711 0.23909886]
0.999997581506055
3
[0.02503966 0.25132976 0.25072027 0.23383587 0.23907445]
[0.02528393 0.25189968 0.24920543 0.23405073 0.23956024]
0.9999982467326796
4
[0.02523164 0.25165521 0.25055237 0.23445183 0.23810895]
[0.02555231 0.25130622 0.251512   0.23550955 0.23611993]
0.9999962939415873
0
[0.02530264 0.24972768 0.2536879  0.23293411 0.23834767]
[0.02482166 0.24798853 0.2603505  0.23067527 0.23616404]
0.9999704732922707
1
[0.02558464 0.24965469 0.2510898  0.23420499 0.23946588]
[0.0251487  0.24989035 0.25074431 0.23483635 0.2393803 ]
0.9999987599543787
2
[0.02579613 0.25099289 0.25014191 0.23442633 0.2386427

0
[0.02523643 0.25070678 0.25352256 0.23227417 0.23826006]
[0.0243439  0.24910692 0.25798886 0.23187509 0.23668523]
0.9999835617284856
1
[0.02549355 0.24968675 0.25101614 0.23434104 0.23946251]
[0.02515257 0.2494822  0.25089107 0.23476768 0.23970649]
0.9999992692608727
2
[0.02580938 0.2509456  0.25002806 0.23434099 0.23887597]
[0.02646758 0.25116896 0.24956493 0.23395999 0.23883854]
0.9999977177317976
3
[0.02515159 0.25120535 0.25063675 0.23418115 0.23882516]
[0.02517953 0.25107515 0.24990183 0.23470442 0.23913907]
0.9999995203832991
4
[0.02546381 0.25123361 0.25064953 0.23425145 0.2384016 ]
[0.0253442  0.25180498 0.25096091 0.23509914 0.23679077]
0.9999979711642383
0
[0.02515984 0.25072534 0.25405858 0.23170682 0.23834943]
[0.02482166 0.24879475 0.2583146  0.23214926 0.23591974]
0.9999855068671577
1
[0.02545475 0.24985272 0.2510919  0.23430979 0.23929085]
[0.02510965 0.24960708 0.25078751 0.23487456 0.2396212 ]
0.9999991078965915
2
[0.02579915 0.25101411 0.25010846 0.234193   0.238885

0
[0.02480634 0.25088331 0.25384081 0.2317961  0.23867344]
[0.02588304 0.24950596 0.25548878 0.23188052 0.23724171]
0.9999909203273015
1
[0.02548895 0.24987489 0.25110165 0.23413434 0.23940017]
[0.02529988 0.24980423 0.25061167 0.23475688 0.23952733]
0.9999994868443624
2
[0.02577583 0.25139825 0.25006342 0.23392694 0.23883556]
[0.02600731 0.25067712 0.25020798 0.23407326 0.23903434]
0.9999994398109864
3
[0.02524417 0.25105977 0.25063087 0.23393105 0.23913415]
[0.02517211 0.25093071 0.24998339 0.23437817 0.23953562]
0.9999995656943967
4
[0.02545356 0.25148219 0.25059888 0.23418058 0.23828479]
[0.02529885 0.25107543 0.25037088 0.2350131  0.23824173]
0.9999994035809664
0.28273233698425115 [ 1. -8.  1.]
0
[0.02542515 0.25004003 0.25562861 0.23130681 0.2375994 ]
[0.02472393 0.24818398 0.26039936 0.22958403 0.2371087 ]
0.9999830565325551
1
[0.02556556 0.24963149 0.25132429 0.23357115 0.23990751]
[0.02535637 0.25039375 0.25112145 0.23527496 0.23785346]
0.9999957184752554
2
[0.02594877 0.25123

0
[0.02483296 0.25091006 0.25383082 0.23185634 0.23856983]
[0.02575151 0.2491849  0.25593668 0.23184942 0.23727749]
0.9999912896241292
1
[0.02549542 0.24986234 0.25110208 0.23412758 0.23941259]
[0.02526243 0.2497594  0.25070821 0.23478862 0.23948134]
0.9999994146015553
2
[0.02578729 0.25138328 0.25004497 0.23390491 0.23887956]
[0.02601391 0.25059585 0.25032485 0.23403209 0.2390333 ]
0.9999993832007589
3
[0.02523481 0.25112399 0.2505906  0.23392309 0.2391275 ]
[0.02514448 0.25098272 0.25003179 0.23443026 0.23941075]
0.9999996144105632
4
[0.02544964 0.25141726 0.25058398 0.23420967 0.23833945]
[0.02532559 0.25094443 0.2504428  0.23499651 0.23829067]
0.999999471920834
0
[0.02480634 0.25088331 0.25384081 0.2317961  0.23867344]
[0.02588304 0.24950596 0.25548878 0.23188052 0.23724171]
0.9999909203273015
1
[0.02548895 0.24987489 0.25110165 0.23413434 0.23940017]
[0.02529988 0.24980423 0.25061167 0.23475688 0.23952733]
0.9999994868443624
2
[0.02577583 0.25139825 0.25006342 0.23392694 0.2388355

In [342]:
pars_in = pars_ref[0]
output = fmin(loss_sd2_hist, pars_in, args=(X_ref[0], pars_ref[0], beta_ref[0], hist_ref[0], hist_targ))

[0.02217415 0.22772579 0.23067028 0.20984925 0.21857788 0.09100265]
[0.02270365 0.22677667 0.23756009 0.20901    0.21680525 0.08714434]
0.9999494781124738
[0.02190257 0.22855006 0.23098473 0.2087677  0.2192735  0.09052143]
[0.02252501 0.2255749  0.23656944 0.21022801 0.21531116 0.08979148]
0.9999652265738994
[0.02192957 0.22885657 0.2298603  0.20931562 0.21942597 0.09061196]
[0.02247629 0.22613789 0.23538392 0.21078559 0.21514876 0.09006756]
0.9999656400159949
[0.0218049  0.22840474 0.2295792  0.20967534 0.21920442 0.0913314 ]
[0.02237073 0.22632194 0.23453131 0.2113161  0.21514876 0.09031116]
0.9999700941501011
[0.02198929 0.2283185  0.22937813 0.20968154 0.21932877 0.09130377]
[0.02261271 0.22667273 0.23490971 0.21121216 0.21472002 0.08987268]
0.9999634047547733
[0.02195022 0.22826577 0.22925521 0.20999854 0.21934815 0.09118211]
[0.0227632  0.22665757 0.23452319 0.21065567 0.21503508 0.09036529]
0.9999680410240857
[0.0219509  0.22823355 0.22920416 0.21027806 0.21917277 0.09116056]
[0

[0.02208611 0.22785799 0.22891239 0.21091191 0.21919721 0.09103438]
[0.02323633 0.2270495  0.23325971 0.21066    0.21568468 0.09010978]
0.999973808418559
0.13579296089228238 [-8.      -2.08725]
[0.02213495 0.22771581 0.23070543 0.20992727 0.21847736 0.09103918]
[0.02270365 0.22677667 0.23756009 0.20901    0.21680525 0.08714434]
0.9999492305790411
[0.02189185 0.22851822 0.23101472 0.20882476 0.21920673 0.09054372]
[0.02252501 0.2255749  0.23656944 0.21022801 0.21531116 0.08979148]
0.9999657857118613
[0.02191057 0.22883855 0.22991156 0.20930128 0.21941469 0.09062335]
[0.02247629 0.22613789 0.23538392 0.21078559 0.21514876 0.09006756]
0.99996589092864
[0.02178151 0.22834083 0.22963805 0.20971524 0.21918079 0.09134358]
[0.02237073 0.22632194 0.23453131 0.2113161  0.21514876 0.09031116]
0.9999705499439558
[0.02198045 0.22826404 0.22942133 0.20969536 0.21929845 0.09134037]
[0.02261271 0.22667273 0.23490971 0.21121216 0.21472002 0.08987268]
0.9999637354556729
[0.02194483 0.22823903 0.2293115 

[0.02197795 0.22824361 0.22943268 0.20970838 0.21928707 0.09135031]
[0.02261271 0.22667273 0.23490971 0.21121216 0.21472002 0.08987268]
0.9999638632831296
[0.0219414  0.22823042 0.22932672 0.21000346 0.21927903 0.09121897]
[0.0227632  0.22665757 0.23452319 0.21065567 0.21503508 0.09036529]
0.9999686850174478
[0.02194904 0.22821523 0.22924468 0.21028762 0.2191125  0.09119093]
[0.02261549 0.22644258 0.23427031 0.21059225 0.2153158  0.09076356]
0.9999735558816368
[0.021968   0.22814907 0.22929225 0.21027336 0.2192249  0.09109242]
[0.02294725 0.2262042  0.23400351 0.21066243 0.21563596 0.09054664]
0.9999726980952441
[0.02201316 0.22800818 0.22904358 0.21073538 0.21914232 0.09105738]
[0.02307176 0.22660344 0.23365164 0.21060514 0.21566122 0.0904068 ]
0.9999736657155279
[0.02208465 0.22784112 0.22893088 0.210922   0.21918007 0.09104128]
[0.02323633 0.2270495  0.23325971 0.21066    0.21568468 0.09010978]
0.9999739397868801
0.13485252066070746 [-8.  1.]
[0.02211458 0.22770379 0.23073639 0.2099

[0.02190216 0.22882424 0.22994391 0.20929525 0.21940971 0.09062475]
[0.02247629 0.22613789 0.23538392 0.21078559 0.21514876 0.09006756]
0.9999660810754702
[0.02177485 0.22831372 0.22965897 0.20973404 0.21917359 0.09134483]
[0.02237073 0.22632194 0.23453131 0.2113161  0.21514876 0.09031116]
0.9999707404714702
[0.02197795 0.22824361 0.22943268 0.20970838 0.21928707 0.09135031]
[0.02261271 0.22667273 0.23490971 0.21121216 0.21472002 0.08987268]
0.9999638632831296
[0.0219414  0.22823042 0.22932672 0.21000346 0.21927903 0.09121897]
[0.0227632  0.22665757 0.23452319 0.21065567 0.21503508 0.09036529]
0.9999686850174478
[0.02194904 0.22821523 0.22924468 0.21028762 0.2191125  0.09119093]
[0.02261549 0.22644258 0.23427031 0.21059225 0.2153158  0.09076356]
0.9999735558816368
[0.021968   0.22814907 0.22929225 0.21027336 0.2192249  0.09109242]
[0.02294725 0.2262042  0.23400351 0.21066243 0.21563596 0.09054664]
0.9999726980952441
[0.02201316 0.22800818 0.22904358 0.21073538 0.21914232 0.09105738]
[0

[0.02188371 0.22849828 0.23104032 0.2088437  0.21919239 0.0905416 ]
[0.02252501 0.2255749  0.23656944 0.21022801 0.21531116 0.08979148]
0.9999660436490166
[0.02190216 0.22882424 0.22994391 0.20929525 0.21940971 0.09062475]
[0.02247629 0.22613789 0.23538392 0.21078559 0.21514876 0.09006756]
0.9999660810754702
[0.02177485 0.22831372 0.22965897 0.20973404 0.21917359 0.09134483]
[0.02237073 0.22632194 0.23453131 0.2113161  0.21514876 0.09031116]
0.9999707404714702
[0.02197795 0.22824361 0.22943268 0.20970838 0.21928707 0.09135031]
[0.02261271 0.22667273 0.23490971 0.21121216 0.21472002 0.08987268]
0.9999638632831296
[0.0219414  0.22823042 0.22932672 0.21000346 0.21927903 0.09121897]
[0.0227632  0.22665757 0.23452319 0.21065567 0.21503508 0.09036529]
0.9999686850174478
[0.02194904 0.22821523 0.22924468 0.21028762 0.2191125  0.09119093]
[0.02261549 0.22644258 0.23427031 0.21059225 0.2153158  0.09076356]
0.9999735558816368
[0.021968   0.22814907 0.22929225 0.21027336 0.2192249  0.09109242]
[0

[0.02188371 0.22849828 0.23104032 0.2088437  0.21919239 0.0905416 ]
[0.02252501 0.2255749  0.23656944 0.21022801 0.21531116 0.08979148]
0.9999660436490166
[0.02190216 0.22882424 0.22994391 0.20929525 0.21940971 0.09062475]
[0.02247629 0.22613789 0.23538392 0.21078559 0.21514876 0.09006756]
0.9999660810754702
[0.02177485 0.22831372 0.22965897 0.20973404 0.21917359 0.09134483]
[0.02237073 0.22632194 0.23453131 0.2113161  0.21514876 0.09031116]
0.9999707404714702
[0.02197795 0.22824361 0.22943268 0.20970838 0.21928707 0.09135031]
[0.02261271 0.22667273 0.23490971 0.21121216 0.21472002 0.08987268]
0.9999638632831296
[0.0219414  0.22823042 0.22932672 0.21000346 0.21927903 0.09121897]
[0.0227632  0.22665757 0.23452319 0.21065567 0.21503508 0.09036529]
0.9999686850174478
[0.02194904 0.22821523 0.22924468 0.21028762 0.2191125  0.09119093]
[0.02261549 0.22644258 0.23427031 0.21059225 0.2153158  0.09076356]
0.9999735558816368
[0.021968   0.22814907 0.22929225 0.21027336 0.2192249  0.09109242]
[0

[0.02201316 0.22800818 0.22904358 0.21073538 0.21914232 0.09105738]
[0.02307176 0.22660344 0.23365164 0.21060514 0.21566122 0.0904068 ]
0.9999736657155279
[0.02208465 0.22784112 0.22893088 0.210922   0.21918007 0.09104128]
[0.02323633 0.2270495  0.23325971 0.21066    0.21568468 0.09010978]
0.9999739397868801
0.13485252066070746 [-8.  1.]
[0.02211458 0.22770379 0.23073639 0.20994903 0.21844492 0.09105129]
[0.02270365 0.22677667 0.23756009 0.20901    0.21680525 0.08714434]
0.9999492446036857
[0.02188371 0.22849828 0.23104032 0.2088437  0.21919239 0.0905416 ]
[0.02252501 0.2255749  0.23656944 0.21022801 0.21531116 0.08979148]
0.9999660436490166
[0.02190216 0.22882424 0.22994391 0.20929525 0.21940971 0.09062475]
[0.02247629 0.22613789 0.23538392 0.21078559 0.21514876 0.09006756]
0.9999660810754702
[0.02177485 0.22831372 0.22965897 0.20973404 0.21917359 0.09134483]
[0.02237073 0.22632194 0.23453131 0.2113161  0.21514876 0.09031116]
0.9999707404714702
[0.02197795 0.22824361 0.22943268 0.2097

[0.02188371 0.22849828 0.23104032 0.2088437  0.21919239 0.0905416 ]
[0.02252501 0.2255749  0.23656944 0.21022801 0.21531116 0.08979148]
0.9999660436490166
[0.02190216 0.22882424 0.22994391 0.20929525 0.21940971 0.09062475]
[0.02247629 0.22613789 0.23538392 0.21078559 0.21514876 0.09006756]
0.9999660810754702
[0.02177485 0.22831372 0.22965897 0.20973404 0.21917359 0.09134483]
[0.02237073 0.22632194 0.23453131 0.2113161  0.21514876 0.09031116]
0.9999707404714702
[0.02197795 0.22824361 0.22943268 0.20970838 0.21928707 0.09135031]
[0.02261271 0.22667273 0.23490971 0.21121216 0.21472002 0.08987268]
0.9999638632831296
[0.0219414  0.22823042 0.22932672 0.21000346 0.21927903 0.09121897]
[0.0227632  0.22665757 0.23452319 0.21065567 0.21503508 0.09036529]
0.9999686850174478
[0.02194904 0.22821523 0.22924468 0.21028762 0.2191125  0.09119093]
[0.02261549 0.22644258 0.23427031 0.21059225 0.2153158  0.09076356]
0.9999735558816368
[0.021968   0.22814907 0.22929225 0.21027336 0.2192249  0.09109242]
[0

In [59]:
pars_ref[:], pars_list

([array([-2.97, -7.  , -4.  ])],
 [array([ 0.  , -2.97, -7.  , -1.05, -4.  ,  0.  , -0.78, -0.06, -0.48,
          0.  , -0.24, -0.17,  0.  ,  0.04,  0.  ])])

In [362]:
loss_sd2_hist(np.array([-5.44, -5.06]), X_ref[0], pars_ref[0], beta_ref[0], hist_ref[0], hist_targ)

0
[0.02217415 0.22772579 0.23067028 0.20984925 0.21857788 0.09100265]
[0.02270365 0.22677667 0.23756009 0.20901    0.21680525 0.08714434]
0.9999494781124738
1
[0.02329492 0.22705077 0.22802042 0.21291712 0.21757921 0.09113758]
[0.02334638 0.2283862  0.22862152 0.21506246 0.21700799 0.08757544]
0.9999781808303895
2
[0.0235626  0.22837096 0.22623684 0.21267452 0.21806123 0.09109384]
[0.02424319 0.22799004 0.22757571 0.21341231 0.21787711 0.08890163]
0.9999894966391973
3
[0.02273943 0.22782826 0.22832975 0.2128583  0.21732499 0.09091928]
[0.02299438 0.22970954 0.22732065 0.21343812 0.21862765 0.08790967]
0.9999833183191307
4
[0.0232005  0.2276179  0.22812949 0.21273592 0.21750473 0.09081146]
[0.02346861 0.22945888 0.229518   0.21476839 0.21448321 0.08830291]
0.9999802250176928
5
[0.02305648 0.22760919 0.22813931 0.21223337 0.21809371 0.09086794]
[0.02312967 0.22453915 0.22450602 0.20746311 0.21409643 0.10626563]
0.9996635403132106
0
[0.02190257 0.22855006 0.23098473 0.2087677  0.2192735  

6.574445524761564

In [363]:
X_ref[0][-1], X_ref[1][-2]

(array([141, 105]), array([161, 104]))

In [364]:
# loss_sd2_hist(np.array([-7., -4.]), X_ref[0], pars_ref[0], beta_ref[0], hist_ref[0], hist_targ)

0
[0.02216507 0.2277279  0.23067004 0.2098804  0.21854046 0.09101612]
[0.02270365 0.22677667 0.23756009 0.20901    0.21680525 0.08714434]
0.9999493186530525
1
[0.02329583 0.22703052 0.22803246 0.21292995 0.21757022 0.09114101]
[0.02334638 0.2283862  0.22862152 0.21506246 0.21700799 0.08757544]
0.9999781632938969
2
[0.02356245 0.22836014 0.22625112 0.21267451 0.21804447 0.09110732]
[0.02424319 0.22799004 0.22757571 0.21341231 0.21787711 0.08890163]
0.999989442502041
3
[0.02273402 0.22782947 0.22832582 0.21287246 0.21733    0.09090823]
[0.02299438 0.22970954 0.22732065 0.21343812 0.21862765 0.08790967]
0.9999834189373424
4
[0.0232009  0.22761029 0.22813811 0.21273997 0.21751972 0.090791  ]
[0.02346861 0.22945888 0.229518   0.21476839 0.21448321 0.08830291]
0.9999803228383659
5
[0.02305554 0.22757641 0.22819622 0.21223844 0.21806621 0.09086718]
[0.02312967 0.22453915 0.22450602 0.20746311 0.21409643 0.10626563]
0.9996634873642769
0
[0.02190213 0.22854497 0.23098382 0.20878826 0.21924384 0

6.57448816626807

In [365]:
loss_sd2_hist(np.array([-7., -4.]), X_ref[1], pars_ref[1], beta_ref[1], hist_ref[1], hist_targ)

0
[0.02151574 0.23008005 0.22879197 0.21111185 0.21813428 0.0903661 ]
[0.02270365 0.22677667 0.23756009 0.20901    0.21680525 0.08714434]
0.9999265680773258
1
[0.02331377 0.22738764 0.22772634 0.2125209  0.21842431 0.09062703]
[0.02334638 0.2283862  0.22862152 0.21506246 0.21700799 0.08757544]
0.9999810148008674
2
[0.02327712 0.22791219 0.22726153 0.21222279 0.21817274 0.09115363]
[0.02424319 0.22799004 0.22757571 0.21341231 0.21787711 0.08890163]
0.9999871089987679
3
[0.02272127 0.22805116 0.22797522 0.21270252 0.21742238 0.09112746]
[0.02299438 0.22970954 0.22732065 0.21343812 0.21862765 0.08790967]
0.9999822452077978
4
[0.02306455 0.22742091 0.22858582 0.21240103 0.21736229 0.09116539]
[0.02346861 0.22945888 0.229518   0.21476839 0.21448321 0.08830291]
0.9999768824814581
5
[0.02309589 0.22739428 0.22752774 0.21287138 0.21797684 0.09113388]
[0.02312967 0.22453915 0.22450602 0.20746311 0.21409643 0.10626563]
0.9996739144847849
0
[0.02217886 0.22880628 0.22997033 0.20942303 0.21826309 

6.716010197274094

In [255]:
loss_sd2_hist(np.array([-7.0, -4.0]), X_ref[0], pars_ref[0], beta_ref[0], hist_ref[0], hist_targ)

[0.02206824 0.22695578 0.2312463  0.2117187  0.21737919 0.0906318 ]
[0.02270365 0.22677667 0.23756009 0.20901    0.21680525 0.08714434]
0.999954813962649
0.0038021500690616243 [-7. -4.]


0.0038021500690616243

## Compare distributions, statistical testing

In [27]:
for i, k in enumerate(targets.keys()):
    if k > 200:
        continue
    print('null')
    get_s2_two(k, targets[k], null_counts[k])
#     print('txinf')
#     get_s2_two(k, targets[k], knn_all['txinf'][i])
#     print('tx700')
#     get_s2_two(k, targets[k], knn_all['tx700'][i])
#     print('tx500')
#     get_s2_two(k, targets[k], knn_all['tx500'][i])
    print('1x700')
    get_s2_two(k, targets[k], knn_all['1x700'][i])

null
0 [ 699 6982 7314 6435 6675 2683] [ 715.82589416 7014.96821189 7014.41020808 6550.05803327 6685.18795716
 2807.54969544] [ -16.82589416  -32.96821189  299.58979192 -115.05803327  -10.18795716
 -124.54969544]
1 0 p-value 0.06477022670197327
1 [ 7044 68908 68979 64888 65475 26423] [ 7014.96821189 68745.45810017 68739.98976066 64189.41989162
 65513.66944494 27513.49459072] [   29.03178811   162.54189983   239.01023934   698.58010838
   -38.66944494 -1090.49459072]
1 1 p-value 7.107498401190488e-05
2 [ 7314 68783 68658 64385 65732 26821] [ 7014.41020808 68739.98976066 68734.52185612 64184.31396097
 65508.4581772  27511.30603698] [ 299.58979192   43.01023934  -76.52185612  200.68603903  223.5418228
 -690.30603698]
1 2 p-value 0.007483190721920082
3 [ 6478 64714 64041 60130 61592 24766] [ 6550.05803327 64189.41989162 64184.31396097 59935.32867318
 61171.8148785  25690.06456246] [ -72.05803327  524.58010838 -143.31396097  194.67132682  420.1851215
 -924.06456246]
1 3 p-value 0.0006887766

0 [ 4874 48802 50489 45386 46404 19561] [ 4854.24251119 49036.24159302 49463.50916255 45295.07379777
 47152.55403803 19714.37889743] [  19.75748881 -234.24159302 1025.49083745   90.92620223 -748.55403803
 -153.37889743]
7 0 p-value 4.547473927550435e-05
1 [ 48353 480116 482301 452122 460587 188540] [ 48802.86746767 479733.07288987 481592.4218316  449892.29081739
 459943.18114301 192055.16585045] [ -449.86746767   382.92711013   708.5781684   2229.70918261
   643.81885699 -3515.16585045]
7 1 p-value 2.045806987248768e-12
2 [ 50254 482122 480433 449732 460025 189285] [ 49216.07160707 481930.18379487 479879.16050305 448696.46534748
 459924.99839741 192204.12035012] [ 1037.92839293   191.81620513   553.83949695  1035.53465252
   100.00160259 -2919.12035012]
7 2 p-value 2.37741912295997e-10
3 [ 45051 451254 449022 421481 429955 175284] [ 45128.73684785 450348.5590962  449225.85317862 419521.29181232
 428408.64825423 179413.91081079] [  -77.73684785   905.4409038   -203.85317862  1959.708187

40 3 p-value 5.902804369896714e-41
4 [ 266366 2628203 2621686 2456121 2495756 1033188] [ 267407.51828643 2620546.77779741 2620338.32708808 2446872.59514003
 2497352.40858295 1048802.37310509] [ -1041.51828643   7656.22220259   1347.67291192   9248.40485997
  -1596.40858295 -15614.37310509]
40 4 p-value 2.6418330111670297e-30
5 [ 109971 1083446 1082970 1012553 1034562  506658] [ 112301.98781761 1100539.78362883 1100452.24147904 1027602.58249849
 1048802.37310509  440461.03147094] [ -2330.98781761 -17093.78362883 -17482.24147904 -15049.58249849
 -14240.37310509  66196.96852906]
40 5 p-value 0.0
1x700
0 [ 29110 279423 285460 260991 266525 110011] [ 27420.21282375 280704.19247102 281640.29419641 260054.30077662
 269358.7809021  112342.2188301 ] [ 1689.78717625 -1281.19247102  3819.70580359   936.69922338
 -2833.7809021  -2331.2188301 ]
40 0 p-value 3.0022190818624524e-38
1 [ 279459 2748873 2753290 2577326 2628427 1081305] [ 278877.20307188 2741901.88062306 2752754.56219961 2567412.75386286

In [ ]:
Al Fe Ni Cr Co
-16.82589416  -32.96821189  299.58979192 -115.05803327  -10.18795716 -124.54969544

In [28]:
 -16.82589416  -32.96821189  299.58979192 -115.05803327  -10.18795716 -124.54969544
20.47253984   11.75252954  194.89341128  -72.22808289   15.09589838 -169.98629614
18.87351129    3.42587269  232.80410678  -64.57453799 -103.62340862 -86.90554415


-24.30820551   -8.80891829  267.59700443 -145.58314801   10.44053525 -99.33726787

18.64444444    5.56239316  226.12991453  -82.77094017  -13.88888889 -153.67692308
18.87351129    3.42587269  232.80410678  -64.57453799 -103.62340862 -86.90554415
30.73242609   44.18882986  213.21814743 -100.79731438 -115.05498589 -72.28710312

-171.15009878  259.44646136 -294.4464846   140.7234631   169.98398605 -104.55732714

SyntaxError: invalid syntax (<ipython-input-28-99c728985025>, line 1)